# Obligatorio Muliti-agentes: CFR y MCTS

---
## Índice

1. [Introducción](##Introducción)
2. [Objetivos del Trabajo](##Objetivos-del-Trabajo)
3. [Setup](##Setup)
   1. [Entorno de Ejecución](###Entorno-de-Ejecución)
   2. [Manejo de Dependencias](###Manejo-de-Dependencias)
   2. [Instalación de Dependencias](###Instalación-de-Dependencias)
4. [Counterfactual Regret Minimization](##Counterfactual-Regret-Minimization)
   1. [Introducción a CFR](###Introducción-a-CFR)
   2. [Implementación de CFR](###Implementación-de-CFR)
   3. [Experimentación con Kuhn Poker](###Experimentación-con-Kuhn-Poker)
5. [Monte Carlo Tree Search](##Monte-Carlo-Tree-Search)
   1. [Implementación de MCTS y Funciones de Evaluación](###Implementación-de-MCTS-y-Funciones-de-Evaluación)
   2. [Experimentación con Leduc Poker](###Experimentación-con-Leduc-Poker)
6. [Parte C - Investigación](##Parte-C---Investigación)
   1. [Tarea de Investigación](###Tarea-de-Investigación)
7. [Conclusiones](##Conclusiones)
8. [Referencias](##Referencias)
9. [Descargo de Responsabilidad](##Descargo-de-Responsabilidad)


## Introducción

En este trabajo, exploramos la implementación y experimentación de algoritmos de aprendizaje avanzados en el contexto de juegos alternados de múltiples jugadores. Nos enfocamos específicamente en dos algoritmos clave: CFR (Counterfactual Regret Minimization) y MCTS (Monte Carlo Tree Search), aplicados a variantes de póker. Nuestro objetivo es no solo implementar estos algoritmos sino también investigar su rendimiento y efectividad en diferentes escenarios de juego.


## Objetivos del Trabajo

El principal objetivo de este trabajo es profundizar en el entendimiento y aplicación práctica de los algoritmos CFR y MCTS en entornos de juego competitivos, así como explorar posibles extensiones y mejoras a estos métodos.



## Setup

### Entorno de Ejecución

[PettingZoo](https://pettingzoo.farama.org/index.html) es una biblioteca de juegos multiagentes que proporciona una amplia variedad de entornos diseñados específicamente para la investigación en aprendizaje por refuerzo. Este entorno será la base de nuestros experimentos, permitiéndonos simular y analizar de manera efectiva las interacciones y estrategias de los agentes en diferentes juegos.

#### Características Clave de PettingZoo

- **Variedad de Juegos**: Ofrece una extensa colección de juegos, incluyendo clásicos y nuevos desafíos, lo que nos permite explorar una amplia gama de escenarios y dinámicas de juego.
- **Compatibilidad con Aprendizaje por Refuerzo**: Diseñado para ser compatible con las técnicas y algoritmos de aprendizaje por refuerzo más comunes, facilitando la integración y experimentación.
- **Entorno Multiagente**: Especialmente orientado a entornos multiagentes, lo que lo hace ideal para estudiar juegos alternados de múltiples jugadores como los que abordaremos.

#### Implementación de los Juegos

En nuestro trabajo, la catedra nos proporcionó una implementación de los juegos Kuhn Poker y Leduc Poker, que utilizaremos para nuestros experimentos. Estas implementaciones se basan en la biblioteca PettingZoo.

1. **Kuhn Poker**: Un juego de póker simplificado que servirá como campo de prueba para el algoritmo CFR. Experimentaremos tanto con versiones de 2 como de 3 jugadores.
2. **Leduc Poker**: Un juego de póker más complejo que será utilizado para probar y evaluar el algoritmo MCTS, así como las funciones de evaluación integradas.

Estos juegos nos proporcionarán una plataforma sólida para implementar, probar y analizar los algoritmos de aprendizaje en un contexto competitivo y controlado.



### Manejo de Dependencias

Para garantizar la consistencia y reproducibilidad de nuestros experimentos, utilizaremos [Conda](https://docs.conda.io/en/latest/), un sistema de gestión de paquetes y entornos, que nos permitirá crear un entorno aislado con todas las dependencias necesarias para nuestro proyecto.

> **Nota**: Para instalar Conda, siga las instrucciones en la [documentación oficial](https://docs.conda.io/projects/conda/en/latest/user-guide/install/index.html).


#### Ventajas de Usar Conda

- **Reproducibilidad**: Conda nos ayuda a garantizar que todos los experimentos se puedan reproducir en diferentes máquinas con las mismas versiones de paquetes y dependencias.
- **Gestión de Dependencias**: Maneja fácilmente las dependencias y paquetes necesarios para el proyecto.
- **Entornos Aislados**: Permite crear entornos aislados para evitar conflictos entre distintos proyectos.


#### Archivo `environment.yml`

En la raíz del proyecto, hemos incluido un archivo `environment.yml` que contiene todas las especificaciones necesarias para crear el entorno de Conda adecuado. Este archivo incluye:

- Las versiones específicas de Python y las bibliotecas necesarias.
- Dependencias adicionales requeridas para ejecutar los algoritmos de aprendizaje por refuerzo y los juegos en PettingZoo.

#### Creación del Entorno con `environment.yml`

Para crear el entorno utilizando este archivo, ejecute el siguiente comando en su terminal:

```bash
conda env create -f environment.yml
```

Una vez creado el entorno, puede activarlo usando:

```bash
conda activate multi-agentes
```


### Instalación de Dependencias

In [1]:
# Entornos
import pettingzoo
from base.game import AlternatingGame, AgentID

# Agentes
from agents.counterfactualregret import CounterFactualRegret
from agents.counterfactualregretv2 import EnhancedCounterFactualRegret
from agents.mcts_t import MonteCarloTreeSearch
from agents.agent_random import RandomAgent

# Juegos
from games.kuhn import KuhnPoker
from games.kuhn3 import KuhnPoker3
from games.leduc import Leduc

# Herramientas
from tqdm import tqdm
from base.utils import run
import numpy as np

### Semilla para Reproducibilidad

In [2]:
SEED = 134
np.random.seed(SEED)

## Counterfactual Regret Minimization

### Introducción a CFR

La técnica de Counterfactual Regret Minimization (CFR) representa un hito fundamental en el campo del aprendizaje por refuerzo, particularmente en juegos de información imperfecta como el póker. CFR es un algoritmo que itera sobre posibles estrategias para minimizar el arrepentimiento contrafactual, es decir, la diferencia entre la recompensa obtenida y la que se hubiera obtenido si se hubiera jugado de manera óptima en cada estado del juego.

#### Fundamentos

El CFR trabaja analizando las decisiones tomadas en cada punto de decisión de un juego y evaluando el "arrepentimiento" de no haber tomado otras alternativas. Esta evaluación se basa en la noción de arrepentimiento contrafactual, que es la diferencia entre el beneficio obtenido y el beneficio que se hubiera obtenido si se hubiera jugado la mejor estrategia posible en retrospectiva.

#### Aplicaciones

Este método ha demostrado ser particularmente poderoso en juegos de póker, donde la información es incompleta y las estrategias óptimas no son siempre claras. La implementación de CFR en juegos como Kuhn Poker y Leduc Poker nos permite explorar cómo este algoritmo puede adaptarse y aprender estrategias efectivas en entornos de decisión complejos y dinámicos.

#### Objetivo de la Sección

En esta sección, nos enfocaremos en la implementación del algoritmo CFR. Detallaremos su estructura, funcionamiento y aplicaremos el algoritmo a juegos de póker de múltiples jugadores, analizando cómo se adapta y evoluciona la estrategia del juego a lo largo de múltiples iteraciones.



### Implementación de CFR

En este proyecto, hemos implementado el algoritmo Counterfactual Regret Minimization (CFR) para experimentar en juegos de póker (aunque podrian ser usados en otros contextos). Esta implementación se centra en desarrollar y refinar estrategias de juego en entornos de información imperfecta. El código completo para esta implementación se puede encontrar en el archivo `/agents/counterfactualregret.py` del proyecto.

#### Descripción de la Implementación

La implementación de CFR en nuestro proyecto consta de varias partes clave:

1. **Clase `Node`**: Representa un nodo en el árbol de juego. Cada nodo contiene información sobre el estado del juego (information set), el agente actual, la observación recibida y mantiene un registro del arrepentimiento acumulado y la política aprendida.

2. **Métodos de Actualización y Estrategia**: Dentro de la clase `Node`, se implementan métodos para actualizar los valores de arrepentimiento contrafactual y para ajustar la estrategia actual basada en estos arrepentimientos.

3. **Clase `CounterFactualRegret`**: Extiende la clase `Agent` (basado en `PettingZoo`) y se utiliza para representar un agente que emplea CFR. Contiene métodos para realizar acciones, entrenar utilizando CFR y realizar el algoritmo de CFR de manera recursiva.

4. **Función `cfr_rec`**: Implementa la lógica del CFR de manera recursiva, calculando la utilidad de cada nodo y actualizando la estrategia y los arrepentimientos.

#### Naturaleza Agnóstica al Juego de las Implementaciones de Agentes

Es importante destacar un aspecto fundamental de nuestras implementaciones de agentes, incluido el agente que utiliza el algoritmo Counterfactual Regret Minimization (CFR). Estos agentes son **agnósticos al juego**, lo que significa que su diseño y funcionamiento no están limitados o definidos por las reglas o acciones específicas de un juego en particular, o por la cantidad de jugadores. En cambio, estos agentes se pueden utilizar en cualquier juego que cumpla con los requisitos básicos de la biblioteca PettingZoo, lo que los hace altamente adaptables y reutilizables.

### Experimentación con Kuhn Poker

[Kuhn Poker](https://en.wikipedia.org/wiki/Kuhn_poker) es un juego de póker simplificado que sirve como un modelo excelente para estudiar algoritmos de aprendizaje por refuerzo en juegos de información imperfecta. Este juego representa una versión reducida del póker tradicional, lo que lo convierte en un entorno ideal para experimentos y análisis en el campo de la teoría de juegos y la inteligencia artificial.


![CFR](_assets/CFR-poker.png)

#### Descripción del Kuhn Poker

- **Jugadores**: El juego clásico involucra a 2 jugadores, aunque puede adaptarse para 3 jugadores.
- **Baraja**: Se utiliza una baraja reducida, típicamente con tres cartas (por ejemplo, un As, un Rey y una Reina).
- **Dinámica del Juego**: Cada jugador recibe una carta y tiene la opción de apostar (bet) o pasar (check). El juego tiene una estructura de apuestas limitada y presenta oportunidades para [farolear](https://es.wikipedia.org/wiki/Farol_(envite)) y realizar estrategias basadas en la información limitada.

#### Experimentación Planeada

En nuestro proyecto, experimentaremos con Kuhn Poker de la siguiente manera:

1. **Con 2 Jugadores**: Inicialmente, nos enfocaremos en la versión clásica de Kuhn Poker con 2 jugadores. Aquí, aplicaremos y evaluaremos el algoritmo de Counterfactual Regret Minimization para desarrollar estrategias efectivas y analizar cómo los agentes aprenden y se adaptan a lo largo de múltiples juegos.

2. **Expansión a 3 Jugadores**: Posteriormente, expandiremos nuestra experimentación para incluir una versión de 3 jugadores del juego. Esta variante presenta desafíos adicionales y complejidades, permitiéndonos explorar cómo los algoritmos se adaptan a un entorno de juego más dinámico y a la presencia de un jugador adicional.

Estas experimentaciones nos permitirán obtener una comprensión más profunda de la efectividad del algoritmo CFR y cómo se puede adaptar a diferentes configuraciones de juego.

### Kuhn Poker con 2 Jugadores

#### CFR vs Random

In [2]:
khun2 = KuhnPoker(render_mode='')
khun2.reset() # Reseteamos el juego

# Creamos los agentes
cfr_agent = CounterFactualRegret(game=khun2, agent=khun2.agents[0])
random_agent = RandomAgent(game=khun2, agent=khun2.agents[1])

agents = {
    khun2.agents[0]: cfr_agent,
    khun2.agents[1]: random_agent
}

In [3]:
# entrenamos
KHUN2_TRAINING_STEPS = 10000
cfr_agent.train(KHUN2_TRAINING_STEPS)

100%|██████████| 10000/10000 [00:16<00:00, 607.09it/s]


In [4]:
# Imrimimos la estrategia de nuestro agente CFR
def print_strategy(agent):
    for n in sorted(agent.node_dict.keys()): 
        print(n + '\t', agent.node_dict[n].agent + '\t', agent.node_dict[n].policy())

print_strategy(cfr_agent)

0	 agent_0	 [0.69924624 0.30075376]
0b	 agent_1	 [9.99851500e-01 1.48500149e-04]
0p	 agent_1	 [0.66378784 0.33621216]
0pb	 agent_0	 [9.99852464e-01 1.47536146e-04]
1	 agent_0	 [0.99593846 0.00406154]
1b	 agent_1	 [0.66067448 0.33932552]
1p	 agent_1	 [0.99538319 0.00461681]
1pb	 agent_0	 [0.37501183 0.62498817]
2	 agent_0	 [0.08510005 0.91489995]
2b	 agent_1	 [1.47623265e-04 9.99852377e-01]
2p	 agent_1	 [1.47623265e-04 9.99852377e-01]
2pb	 agent_0	 [1.50240385e-04 9.99849760e-01]


In [5]:
KHUN2_GAME_ROUNDS = 10_00_000
_, mean_reward = run(khun2, agents, KHUN2_GAME_ROUNDS)

print(mean_reward)

100%|██████████| 1000000/1000000 [01:33<00:00, 10727.45it/s]

0.154535


Podemos observar que la estrategia aprendida por el agente CFR es mucho más efectiva que la estrategia aleatoria ya que en promedio gana más fichas. Esto se debe a que el agente CFR aprende a jugar de manera óptima en cada estado del juego, mientras que el agente aleatorio simplemente toma decisiones aleatorias.

> Cabe destacar que en este caso el agente CFR es siempre el primer jugador, por lo que tiene una [desventaja inicial](https://en.wikipedia.org/wiki/Kuhn_poker#Optimal_strategy). 

#### CFR vs CFR

In [6]:
cfr_agent2 = CounterFactualRegret(game=khun2, agent=khun2.agents[1])

agents = {
    khun2.agents[0]: cfr_agent,
    khun2.agents[1]: cfr_agent2
}

In [7]:
cfr_agent2.train(KHUN2_TRAINING_STEPS)

100%|██████████| 10000/10000 [00:16<00:00, 596.66it/s]


In [8]:
print("\nAgent 1 strategy:\n")
print_strategy(cfr_agent)

print("\nAgent 2 strategy:\n")
print_strategy(cfr_agent2)


Agent 1 strategy:

0	 agent_0	 [0.69924624 0.30075376]
0b	 agent_1	 [9.99851500e-01 1.48500149e-04]
0p	 agent_1	 [0.66378784 0.33621216]
0pb	 agent_0	 [9.99852464e-01 1.47536146e-04]
1	 agent_0	 [0.99593846 0.00406154]
1b	 agent_1	 [0.66067448 0.33932552]
1p	 agent_1	 [0.99538319 0.00461681]
1pb	 agent_0	 [0.37501183 0.62498817]
2	 agent_0	 [0.08510005 0.91489995]
2b	 agent_1	 [1.47623265e-04 9.99852377e-01]
2p	 agent_1	 [1.47623265e-04 9.99852377e-01]
2pb	 agent_0	 [1.50240385e-04 9.99849760e-01]

Agent 2 strategy:

0	 agent_0	 [0.88786281 0.11213719]
0b	 agent_1	 [9.99851764e-01 1.48235992e-04]
0p	 agent_1	 [0.62804647 0.37195353]
0pb	 agent_0	 [9.99848714e-01 1.51285930e-04]
1	 agent_0	 [9.99846861e-01 1.53139357e-04]
1b	 agent_1	 [0.65861866 0.34138134]
1p	 agent_1	 [0.99878308 0.00121692]
1pb	 agent_0	 [0.54095134 0.45904866]
2	 agent_0	 [0.65005519 0.34994481]
2b	 agent_1	 [1.49566258e-04 9.99850434e-01]
2p	 agent_1	 [2.99132516e-04 9.99700867e-01]
2pb	 agent_0	 [1.45645208e-04 

In [9]:
_, mean_reward = run(khun2, agents, KHUN2_GAME_ROUNDS)

print(mean_reward)

100%|██████████| 1000000/1000000 [01:27<00:00, 11398.24it/s]


-0.057136


Podemos concluir que llegamos a un [Equilibrio de Nash](https://es.wikipedia.org/wiki/Equilibrio_de_Nash), donde el primer jugador 1 tiene una recompensa esperada de ~1/18:

> "The game has a mixed-strategy Nash equilibrium; when both players play equilibrium strategies, the first player should expect to lose at a rate of −1/18 per hand (as the game is zero-sum, the second player should expect to win at a rate of +1/18). There is no pure-strategy equilibrium." - [Wikipedia](https://en.wikipedia.org/wiki/Kuhn_poker#Optimal_strategy)

### Kuhn Poker con 3 Jugadores

En esta sección, extendemos nuestra exploración del Kuhn Poker a un escenario más complejo y desafiante: un juego con tres jugadores. El Kuhn Poker de tres jugadores introduce una dinámica de juego adicional y requiere estrategias más sofisticadas, dada la presencia de un jugador extra. Este entorno nos permite investigar cómo los algoritmos de aprendizaje por refuerzo, específicamente el CFR, se adaptan y se desempeñan en un contexto de juego más complejo.

#### CFR vs CFR vs CFR

Para evaluar el rendimiento del algoritmo CFR en este contexto, configuramos un experimento donde tres agentes independientes, cada uno utilizando una instancia del algoritmo CFR, compiten entre sí. 

#### Expectativas de Utilidad: \( u_1 < u_2 < u_3 \)

En términos de resultados, esperamos observar una relación específica en las utilidades obtenidas por los tres agentes, específicamente \( u_1 < u_2 < u_3 \). Esta expectativa se basa en la hipótesis de que el tercer jugador, al ser el último en actuar en cada ronda, tiene una ventaja estratégica al contar con más información que los otros dos jugadores. Del mismo modo, el segundo jugador tendría una ventaja sobre el primero. Esta hipótesis será probada y analizada a través de los resultados obtenidos en nuestras experimentaciones.

Esta fase del proyecto no solo nos proporcionará insights sobre el rendimiento del CFR en un escenario de tres jugadores, sino que también ampliará nuestra comprensión sobre la dinámica de los juegos de información imperfecta y las estrategias óptimas en dichos contextos.


In [48]:
khun3 = KuhnPoker3(render_mode='')
khun3.reset() # Reseteamos el juego

# Creamos los agentes
cfr_agent1 = CounterFactualRegret(game=khun3, agent=khun3.agents[0])
cfr_agent2 = CounterFactualRegret(game=khun3, agent=khun3.agents[1])
cfr_agent3 = CounterFactualRegret(game=khun3, agent=khun3.agents[2])


agents = {
    khun3.agents[0]: cfr_agent1,
    khun3.agents[1]: cfr_agent2,
    khun3.agents[2]: cfr_agent3
}

In [50]:
# entrenamos
KHUN3_TRAINING_STEPS = 50_000
cfr_agent1.train(KHUN3_TRAINING_STEPS)
cfr_agent2.train(KHUN3_TRAINING_STEPS)
cfr_agent3.train(KHUN3_TRAINING_STEPS)

100%|██████████| 50000/50000 [06:23<00:00, 130.47it/s]


In [51]:
print("\nAgent 1 strategy:\n")
print_strategy(cfr_agent1)

print("\nAgent 2 strategy:\n")
print_strategy(cfr_agent2)

print("\nAgent 3 strategy:\n")
print_strategy(cfr_agent3)


Agent 1 strategy:

0	 agent_0	 [0.99795265 0.00204735]
0b	 agent_1	 [9.99976895e-01 2.31053604e-05]
0bb	 agent_2	 [9.99976865e-01 2.31352952e-05]
0bp	 agent_2	 [9.99976865e-01 2.31352952e-05]
0p	 agent_1	 [0.9307425 0.0692575]
0pb	 agent_2	 [9.99976865e-01 2.31352952e-05]
0pbb	 agent_0	 [9.9997680e-01 2.3199703e-05]
0pbp	 agent_0	 [9.99930401e-01 6.95991091e-05]
0pp	 agent_2	 [0.82512136 0.17487864]
0ppb	 agent_0	 [9.99860802e-01 1.39198218e-04]
0ppbb	 agent_1	 [9.99630314e-01 3.69685767e-04]
0ppbp	 agent_1	 [9.99815157e-01 1.84842884e-04]
1	 agent_0	 [0.99430878 0.00569122]
1b	 agent_1	 [9.99873700e-01 1.26299962e-04]
1bb	 agent_2	 [9.99977028e-01 2.29716071e-05]
1bp	 agent_2	 [9.99977028e-01 2.29716071e-05]
1p	 agent_1	 [0.77882365 0.22117635]
1pb	 agent_2	 [0.99683408 0.00316592]
1pbb	 agent_0	 [9.99977004e-01 2.29959067e-05]
1pbp	 agent_0	 [0.98723015 0.01276985]
1pp	 agent_2	 [0.69007284 0.30992716]
1ppb	 agent_0	 [0.99868436 0.00131564]
1ppbb	 agent_1	 [9.99976898e-01 2.31021577

In [61]:
KHUN3_GAME_ROUNDS = 5_000_000

cum_rewards = dict(map(lambda ag: (ag, 0.), khun3.agents))

for _ in tqdm(range(KHUN3_GAME_ROUNDS)):
    khun3.reset()
    turn = 0
    while not khun3.done():
        agent = agents[khun3.agent_selection]
        a = agent.action()
        khun3.step(action=a)
        turn += 1
    for ag in khun3.agents:
        cum_rewards[ag] += khun3.rewards[ag]

print('Average rewards:', dict(map(lambda ag: (ag, cum_rewards[ag]/KHUN3_GAME_ROUNDS), khun3.agents)))

100%|██████████| 5000000/5000000 [07:58<00:00, 10441.72it/s]

Average rewards: {'agent_0': -0.0260434, 'agent_1': -0.019646, 'agent_2': 0.0456894}


Como podemos observar, los resultados obtenidos en esta experimentación confirman nuestra hipótesis inicial: el tercer jugador obtiene la mayor utilidad, seguido del segundo jugador y finalmente el primer jugador.

> "A family of Nash equilibria for 3-player Kuhn poker is known analytically, which makes it the largest game with more than two players with analytic solution. The family is parameterized using 4–6 parameters (depending on the chosen equilibrium). In all equilibria, player 1 has a fixed strategy, and he always checks as the first action; player 2's utility is constant, equal to –1/48 per hand. The discovered equilibrium profiles show an interesting feature: by adjusting a strategy parameter $\beta$ (between 0 and 1), player 2 can freely shift utility between the other two players while still remaining in equilibrium; player 1's utility is equal to $-\frac{1+2\beta}{48}$ (which is always worse than player 2's utility), player 3's utility is $\frac{1+\beta}{24}$." - [Wikipedia](https://en.wikipedia.org/wiki/Kuhn_poker#3-player_Kuhn_Poker)

En nuestro caso no tenemos un $\beta$ para parametrizar la estrategia del jugador 2, pero podemos ver que el jugador 2 tiene una utilidad similar a ~-1/48 por mano, que el jugador 3 tiene un poco mayor a 1/24 por mano, y que el jugador 1 tiene una utilidad un poco menor a -1/48 por mano. Lo cual es consistente con lo que se describe en trabajos previos.

In [57]:
khun2 = KuhnPoker(render_mode='')
khun2.reset() # Reseteamos el juego
print(khun2.observe(khun2.agents[0]))
mcts_estimador = MonteCarloTreeSearch(khun2, khun2.agents[0], 100, 100)
v = mcts_estimador.estimate()
print(v)

1
0.9982


In [43]:
khun2 = KuhnPoker(render_mode='')
khun2.reset() # Reseteamos el juego


def estimator(game: AlternatingGame, agent: AgentID):
    mcts_estimador = MonteCarloTreeSearch(game, agent)
    v = mcts_estimador.estimate()
    return v

# Creamos los agentes
cfr_agent = EnhancedCounterFactualRegret(game=khun2, agent=khun2.agents[0], value_estimator=estimator, max_depth=1)
random_agent = RandomAgent(game=khun2, agent=khun2.agents[1])

agents = {
    khun2.agents[0]: cfr_agent,
    khun2.agents[1]: random_agent
}

In [3]:
cfr_agent.train(50)

100%|██████████| 50/50 [00:15<00:00,  3.24it/s]


In [4]:
# Imrimimos la estrategia de nuestro agente CFR
def print_strategy(agent):
    for n in sorted(agent.node_dict.keys()): 
        print(n + '\t', agent.node_dict[n].agent + '\t', agent.node_dict[n].policy())

print_strategy(cfr_agent)

0	 agent_0	 [0.03333333 0.96666667]
1	 agent_0	 [0.02631579 0.97368421]
2	 agent_0	 [0.02631579 0.97368421]


## Referencias

- [PettingZoo: Gym for Multi-Agent Reinforcement Learning](https://arxiv.org/abs/2009.14471)
- [Regret Circuits: Composability of Regret Minimizers](https://blog.ml.cmu.edu/2019/08/02/regret-circuits-composability-of-regret-minimizers/)
- [Khun Poker](https://en.wikipedia.org/wiki/Kuhn_poker)
- [Leduc Poker](https://en.wikipedia.org/wiki/Leduc_poker)
- [Equilibrio de Nash](https://es.wikipedia.org/wiki/Equilibrio_de_Nash)
- [Farol (envite)](https://es.wikipedia.org/wiki/Farol_(envite))

## Descargo de Responsabilidad

### Participación de la Inteligencia Artificial en la Composición del Texto

Este documento incluye secciones cuya composición textual ha sido asistida por una Inteligencia Artificial (IA). Es importante destacar que, aunque la IA ha contribuido en la redacción de ciertas partes del texto para mejorar la claridad y estructura del mismo, los siguientes elementos son enteramente responsabilidad del alumno:

- **Desarrollo del Código**: Todo el código presente en este trabajo ha sido desarrollado y escrito por el alumno, reflejando su comprensión y aplicación práctica de los conceptos aprendidos.
- **Conclusiones**: Las conclusiones extraídas de la experimentación y análisis son fruto del criterio y razonamiento del alumno, basadas en los resultados obtenidos y las observaciones realizadas durante el desarrollo del proyecto.
- **Decisiones Metodológicas y Conceptuales**: Todas las decisiones relacionadas con la metodología, enfoque del proyecto, y la interpretación de los conceptos teóricos son producto del trabajo independiente del alumno.

### Propósito de la Asistencia de IA

El uso de la IA se ha limitado a proporcionar asistencia en la redacción para facilitar una comunicación clara y efectiva. En ningún momento, la IA ha influido en las decisiones técnicas, analíticas o conceptuales que conforman la esencia y los resultados del proyecto.